In [102]:
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

In [103]:
# === STEP 1: Load & Clean Data ===
df = pd.read_csv('/content/0000-1 (1).csv')

In [104]:
# Drop unwanted index column
if 'Unnamed: 0' in df.columns:
    df.drop(columns=['Unnamed: 0'], inplace=True)

In [105]:
# Convert 'explicit' to numeric
df['explicit'] = df['explicit'].astype(str).map({'true': 1, 'false': 0})
df['explicit'] = df['explicit'].fillna(0)

In [106]:
# Drop rows with missing values in key features
df.dropna(subset=[
    'popularity', 'duration_ms', 'explicit', 'danceability', 'energy',
    'key', 'loudness', 'mode', 'speechiness', 'acousticness',
    'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature'
], inplace=True)

In [107]:
# Select features for recommendation
feature_cols = [
    'popularity', 'duration_ms', 'explicit', 'danceability', 'energy',
    'key', 'loudness', 'mode', 'speechiness', 'acousticness',
    'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature'
]

df_features = df[feature_cols].copy()
df_features.fillna(df_features.mean(numeric_only=True), inplace=True)

In [108]:
# === STEP 2: Scale Data ===
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df_features)

In [109]:
# === STEP 3: Get User Input ===
# Replace these values with real user input or dynamic input logic
user_input = {
    'popularity': 70,
    'duration_ms': 210000,
    'explicit': 0,
    'danceability': 0.8,
    'energy': 0.7,
    'key': 5,
    'loudness': -5.0,
    'mode': 1,
    'speechiness': 0.05,
    'acousticness': 0.3,
    'instrumentalness': 0.0,
    'liveness': 0.1,
    'valence': 0.6,
    'tempo': 120.0,
    'time_signature': 4
}

user_vector = [user_input[col] for col in feature_cols]
user_scaled = scaler.transform([user_vector])

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [110]:
# === STEP 4: Compute Cosine Similarity ===
similarities = cosine_similarity(user_scaled, scaled_features)[0]
top_indices = np.argsort(similarities)[::-1][:5]

In [111]:
# === STEP 5: Show Recommendations ===
recommendations = df.iloc[top_indices][['track_name', 'artists', 'track_genre', 'popularity']]
print("🎧 Top 5 Recommended Tracks:")
print(recommendations.to_string(index=False))

🎧 Top 5 Recommended Tracks:
               track_name             artists track_genre  popularity
             KESI - Remix Camilo;Shawn Mendes   reggaeton          76
             KESI - Remix Camilo;Shawn Mendes      reggae          76
Mafiosa (feat. Caroliina)  Lartiste;Caroliina      french          65
                Yes or No              Brando  deep-house          64
                   So Bad             高爾宣 OSN    mandopop          56


In [112]:
%%writefile my_app.py

import streamlit as st
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

@st.cache_data
def load_data():
    df = pd.read_csv('/content/0000-1 (1).csv')

    # Drop unwanted index column if it exists
    if 'Unnamed: 0' in df.columns:
        df.drop(columns=['Unnamed: 0'], inplace=True)

    # Drop rows with missing values in key features
    df.dropna(subset=[
        'popularity', 'duration_ms', 'explicit', 'danceability', 'energy',
        'key', 'loudness', 'mode', 'speechiness', 'acousticness',
        'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature'
    ], inplace=True)

    # Convert 'explicit' to numeric
    df['explicit'] = df['explicit'].astype(str).map({'true': 1, 'false': 0})
    df['explicit'] = df['explicit'].fillna(0)

    # Drop non-numeric columns
    df_numeric = df.drop(columns=['track_id', 'track_name', 'artists', 'album_name', 'track_genre'])

    # Fill any unexpected NaNs with column means
    df_numeric = df_numeric.fillna(df_numeric.mean(numeric_only=True))

    # Scale
    scaler = StandardScaler()
    scaled = scaler.fit_transform(df_numeric)

    return df, df_numeric.columns.tolist(), scaler, scaled


# Load data
df, feature_cols, scaler, scaled_features = load_data()

# App Title
st.title("🎧Spotify Song Recommender")
st.markdown("Adjust the audio features below to get personalized song suggestions.")

# Collect user inputs
user_inputs = []
for col in feature_cols:
    if col == 'explicit':
        val = st.selectbox(f'{col}', [0, 1], index=0)
    elif col in ['key', 'mode', 'time_signature']:
        val = st.number_input(f'{col} (integer)', value=0, step=1)
    elif col == 'tempo':
        val = st.slider(f'{col} (BPM)', 40.0, 220.0, 120.0)
    elif col == 'loudness':
        val = st.slider(f'{col} (dB)', -60.0, 0.0, -10.0)
    else:
        val = st.slider(f'{col}', 0.0, 1.0, 0.5)

    # Ensure no NaN values
    if pd.isna(val):
        val = 0.0
    user_inputs.append(val)

# Recommend songs
if st.button("Recommend🎧"):
    try:
        user_scaled = scaler.transform([user_inputs])
        similarities = cosine_similarity(user_scaled, scaled_features)[0]
        indices = np.argsort(similarities)[::-1][:5]
        results = df.iloc[indices][['track_name', 'artists', 'track_genre', 'popularity']]
        st.subheader("🔥Top Recommendations")
        st.table(results)
    except Exception as e:
        st.error(f"Error: {e}")


Overwriting my_app.py


In [113]:
!wget -q -O - ipv4.icanhazip.com

34.125.105.55


In [114]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋npm warn deprecated debug@4.1.1: Debug versions >=3.2.0 <3.2.7 || >=4 <4.3.1 have a low-severity ReDos regression when used in a Node.js environment. It is recommended you upgrade to 3.2.7 or 4.3.1. (https://github.com/visionmedia/debug/issues/797)
⠋npm warn deprecated axios@0.19.0: Critical security vulnerability fixed in v0.21.1. For more information, see https://github.com/axios/axios/pull/3410
⠋⠙⠹
added 38 packages, removed 74 packages, changed 5 packages, and audited 60 packages in 1s
⠹
⠹5 packages are looking for funding
⠹  run `npm fund` for details
⠹
6 vulnerabilities (1 low, 2 moderate, 3 high)

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠹

In [115]:
!npm audit fix --force

npm warn using --force Recommended protections disabled.
⠙⠹⠸⠼npm warn audit Updating localtunnel to 2.0.2, which is a SemVer major change.
⠼⠴⠦⠧⠇
added 5 packages, removed 42 packages, changed 15 packages, and audited 23 packages in 970ms
⠇
⠇3 packages are looking for funding
⠇  run `npm fund` for details
⠇
# npm audit report

axios  <=0.29.0
Severity: high
Axios Cross-Site Request Forgery Vulnerability - https://github.com/advisories/GHSA-wf5p-g6vw-rhxx
axios Requests Vulnerable To Possible SSRF and Credential Leakage via Absolute URL - https://github.com/advisories/GHSA-jr5f-v2jv-69x6
fix available via `npm audit fix --force`
Will install localtunnel@1.8.3, which is a breaking change
node_modules/axios
  localtunnel  >=1.9.0
  Depends on vulnerable versions of axios
  node_modules/localtunnel

2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force
⠇

In [116]:
! streamlit run my_app.py & npx localtunnel --port 8501



⠙
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.105.55:8501

your url is: https://sad-bikes-beg.loca.lt
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
  Stopping...
^C
